In [437]:
import pandas as pd
from tqdm import tqdm

In [438]:
# TODO 추천에 활용하지 않을 category 목록들을 필터링합니다.
# TODO 필터링 이후, similarity에 대한 BFS 알고리즘을 적용합니다 -> 이전에 구한 similarity가 전체 item을 기준으로 구해졌기 때문에, 예외 처리를 해줘야 합니다.
# TODO BFS 알고리즘을 통해 찾아낸 clustering group을 만들어주고, "ETC"나 "etc"로 group된 item들은 -1로 변환합니다.
# TODO house_interaction에 클러스터 결과를 merge해줍니다. 이때, 클러스터 그룹이 -1이거나 house_interaction에만 포함된 item들은 cluster_id.max() + 1을 더해줍니다.
# TODO model을 학습해서 학습한 결과를 backend에 반영합니다.
# TODO cluster-major-item list를 생성합니다.

In [439]:
item = pd.read_csv("similarity_item_99243.csv")

In [440]:
item.head(1)

,item,category,rating,review,price,preprocessed_title,seller,discount_rate,image,available_product,predict_price,original_title,similarity_list
0,226754,유아·아동|완구·교구|신생아·영아완구,별점 0.0점,리뷰 쓰기첫 리뷰 두 배 적립,NaN,0 개월 한글 영어,FINDEN BEBE(핀데베베),NaN,https://image.ohou.se/i/bucketplace-v2-develop...,미입점,"예상가799,000사이즈정보없음출처smartstore.naver.com",0~36개월/한글영어,[]


In [441]:
del_list = [
"공구",
"욕실용품",
"유아·아동",
"청소용품",
"캠핑·레저",
"반려동물",
"생필품",
"렌탈",
"생활용품",
"식품",
"인테리어시공",
"칼",
"선풍기",
"컵·잔·텀블러",
"조리도구·도마",
"홈갤러리",
"전기요·온수매트",
# "에어컨",
"커피·티용품",
"조명부속품",
"센서등",
"복합기·프린터·스캐너",
"소파·생활커버",
"이미용가전",
"주방잡화",
"그릇·홈세트",
"후크·수납걸이",
"데스크·디자인문구",
"냄비·프라이팬·솥",
"베이킹용품",
# "세탁기·건조기",
"거울",
"매트리스·토퍼",
"파티·이벤트용품",
"해충퇴치기",
"다이어리·플래너",
"수세미걸이·세제통",
"다리미·보풀제거기",
"멀티탭·공유기정리함",
"멀티탭·공유기정리함",
"다리미·보풀제거기",
"디자인문구",
"보드게임",
"리빙박스·바구니",
# "가습기",
"전기히터·온풍기",
"전기·멀티포트"
]

In [442]:
item.category.fillna("|", inplace=True)

In [443]:
import pickle
with open("similarity_99243.pickle", "rb") as pkl:
    data = pickle.load(pkl)

item["similarity_list"] = data

In [444]:
item_ = item[(~item.category.str.contains("|".join(del_list))) & ~(item.category=="|")]

In [445]:
len(item_)

88634

In [446]:
(item_.iloc[6].similarity_list)

[['002 애쉬 인 소파 페 브릭', '002 애쉬 인 소파 페 브릭', 69359, 1.0]]

In [447]:
item_.sort_values("item", inplace=True)

/tmp/ipykernel_26211/1535190289.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  item_.sort_values("item", inplace=True)


In [448]:
list(item_[item_.item == 356808].similarity_list)

[[['쿠키앤크림 사다리 책장 단', '쿠키앤크림 사다리 책장 종 택', 25490, 0.8666666666666667]]]

In [449]:
CLUSTER_ID = 0
NON_CLUSTER_ID = -1

lim = 5
threshold = 0.8

# TODO threshold 이상의 similarity 가지는 아이템을 저장하는 list 칼럼 생성
def get_item(x, threshold):
    tmp = []
    for a in x:
        if a[3] >= threshold:
            tmp.append(a[2])
    return tmp

item_["similar_item_list"] = item_.similarity_list.apply(lambda x:get_item(x, threshold))

/tmp/ipykernel_26211/3209520921.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  item_["similar_item_list"] = item_.similarity_list.apply(lambda x:get_item(x, threshold))


In [450]:
item_[item_.item == 356808]

,item,category,rating,review,price,preprocessed_title,seller,discount_rate,image,available_product,predict_price,original_title,similarity_list,similar_item_list
126402,356808,가구|진열장·책장|책장,별점 4.6점,(685)개 리뷰,"91,100원",쿠키앤크림 사다리 책장 단,채우리,56%,https://image.ohou.se/i/bucketplace-v2-develop...,NaN,NaN,쿠키앤크림 사다리책장 4단 2colors,"[[쿠키앤크림 사다리 책장 단, 쿠키앤크림 사다리 책장 종 택, 25490, 0.8...",[25490]


In [451]:
graph = {item_id:sim_items for item_id, sim_items in zip(item_.item, item_.similar_item_list)}
visited = {item_id:NON_CLUSTER_ID for item_id in item_.item}

In [452]:
# 양방향 그래프로 만들기
for item_id, sim_items in graph.items():
    for item_id_ in sim_items:
        try:
            graph[item_id_].append(item_id)
        except:
            pass

In [453]:
len(graph)

88634

In [454]:
len(visited)

88634

In [455]:
cnt = 0
for g in graph.values():
    if len(g) > 1:
        cnt += 1
cnt

14999

In [456]:
def BFS(item_id:int, cls_id):
    from collections import deque
    q = deque([(item_id, 0)])
    visited[item_id] = cls_id
    while(q):
        node, hop = q.popleft()
        if hop > lim:
            return
        try:
            for id in graph[node]:
                if visited[id] == -1: # 아직 클러스터링되지 않은 아이템인 경우
                    visited[id] = cls_id
                    q.append((id, hop + 1))
        except:
            pass

In [457]:
for item_id in tqdm(graph.keys()):
    if visited[item_id] == NON_CLUSTER_ID and len(graph[item_id]) > 0:
        BFS(item_id, CLUSTER_ID)
        CLUSTER_ID += 1

100%|██████████| 88634/88634 [00:00<00:00, 539827.99it/s]


In [458]:
item_["cls_id"] = item_.item.map(visited)

/tmp/ipykernel_26211/1113527780.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  item_["cls_id"] = item_.item.map(visited)


In [459]:
item_.head(1)

,item,category,rating,review,price,preprocessed_title,seller,discount_rate,image,available_product,predict_price,original_title,similarity_list,similar_item_list,cls_id
2215,1,가구|서랍·수납장|수납장,별점 4.2점,(67)개 리뷰,NaN,66808,DRAWER,NaN,https://image.ohou.se/i/bucketplace-v2-develop...,미입점,예상가정보없음사이즈59.0 × 37.0 × 90.0 (cm)출처www.casa.co.kr,66808,[],[],-1


In [460]:
item_.loc[item_.preprocessed_title.str.contains("etc|ETC"), "cls_id"] = -1

In [461]:
cnt  = 0
for k, v in Counter(visited.values()).items():
    if v == 1:
        cnt += 1
cnt

493

In [462]:
not_clustered_item = []
c = Counter(visited.values())
for k, v in visited.items():
    if c[v] == 1:
        not_clustered_item.append(k)
len(not_clustered_item)

493

In [463]:
for item_id in not_clustered_item:
    # 자신과 이웃인 item의 cluster_group을 찾아서 바꿔주기
    group_candidates = []
    for neighbor_item_id in graph[item_id]:
        try:
            group_candidates.append(visited[neighbor_item_id])
        except:
            pass
    CNT = Counter(group_candidates)
    if len(CNT) >= 2:
        tmp = CNT.most_common(2)
        if tmp[0][0] == -1: # 가장 빈도 높은 클러스터가 클러스터링이 안되어있을 때 : -1
            final_cls_id = tmp[1][0] # 나머지 모든 item을 2등 최빈 클러스터로 묶어줌
            for neighbor_item_id in graph[item_id] + [item_id]:
                visited[neighbor_item_id] = final_cls_id
        else:
            final_cls_id = tmp[0][0] # 나머지 모든 item을 1등 최빈 클러스터로 묶어줌
            for neighbor_item_id in graph[item_id] + [item_id]:
                visited[neighbor_item_id] = final_cls_id
    elif len(CNT) == 1:
        tmp = CNT.most_common(1)
        if tmp[0][0] == -1: # 클러스터가 클러스터링이 안되어있을 때 : -1
            final_cls_id = tmp[1][0] # 나머지 모든 item을 not_clustered_item의 클러스터로 묶어줌
            for neighbor_item_id in graph[item_id] + [item_id]:
                visited[neighbor_item_id] = final_cls_id
        else:
            final_cls_id = tmp[0][0] # 나머지 모든 item을 1등 최빈 클러스터로 묶어줌
            for neighbor_item_id in graph[item_id] + [item_id]:
                visited[neighbor_item_id] = final_cls_id
    else: # 이웃들이 모두 잘려나간 카테고리에 포함되는 경우 -> -1로 처리
        visited[item_id] = -1

In [464]:
not_clustered_item = []
c = Counter(visited.values())
for k, v in visited.items():
    if c[v] == 1:
        not_clustered_item.append(k)
len(not_clustered_item)

2

In [465]:
for item_id in not_clustered_item:
    visited[item_id] = -1

In [466]:
not_clustered_item = []
c = Counter(visited.values())
for k, v in visited.items():
    if c[v] == 1:
        not_clustered_item.append(k)
len(not_clustered_item)

0

In [467]:
# 모든 클러스터를 처리함

In [468]:
len(visited)

88760

In [469]:
len(item_)

88634

In [470]:
# 개수 차이가 나는 이유는 visited의 일부가 잘려나간 카테고리에 포함되기 때문
item_["cls_id"] = item_.item.map(visited)

/tmp/ipykernel_26211/1011874532.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  item_["cls_id"] = item_.item.map(visited)


In [473]:
len(item_[item_.cls_id != -1])
# 전체 88634 item 중에 25039개에 해당하는 아이템을 클러스터링함. (28.2%)

25039

In [478]:
from time import time

now = str(round(time()))[5:]
item_.to_csv(f"clustered_item_{now}.csv", index=False)